In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pyttsx3

# Load the label mapping
label_mapping_path = "./label_mapping.json"
with open(label_mapping_path, 'r') as f:
    label_mapping = json.load(f)

# Reverse the label mapping to get labels from indices
reversed_label_mapping = {v: k for k, v in label_mapping.items()}

# Load the trained model and tokenizer
model_save_path = "./trained_model320"  # Path to the trained model
tokenizer = AutoTokenizer.from_pretrained(model_save_path)
model = AutoModelForSequenceClassification.from_pretrained(model_save_path)

# Set the model to evaluation mode
model.eval()

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Optional: Adjust speech rate
engine.setProperty('rate', 150)  # Change to desired speech rate

# Define a function to make predictions
def predict(texts):
    # Tokenize the input text
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

    # Ensure to run on the appropriate device
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted labels
    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions

# Define a function to speak the predicted answers
def speak(text):
    engine.say(text)
    engine.runAndWait()

# Example input data for testing
test_data = [
    "Tell me about yourself",
]

# Get predictions for the test data
predictions = predict(test_data)

# Map predictions back to labels using the reversed mapping
predicted_labels = [reversed_label_mapping[prediction.item()] for prediction in predictions]

# Print and speak the results
for text, prediction in zip(test_data, predicted_labels):
    # Prepare the speech format
    speech_output = f"You asked: '{text}'. The predicted answer is: '{prediction}'. Would you like to ask anything else?"

    print(f"Question: {text}\nPredicted Answer: {prediction}\n")
    
    # Speak the formatted response
    speak(speech_output)
